# 🧪 ADK Application Testing

This notebook demonstrates how to test an ADK (Agent Development Kit) application.
It covers both local and remote testing, both with Agent Engine and Cloud Run.

> **Note**: This notebook assumes that the agent files are stored in the `app` folder. If your agent files are located in a different directory, please update all relevant file paths and references accordingly.

## Install dependencies

In [ ]:
!pip install google-cloud-aiplatform --upgrade

### Import libraries

In [1]:
import json

import requests
import vertexai

### Initialize Vertex AI Client

In [2]:
# Initialize the Vertex AI client
#LOCATION = "asia-northeast1"
LOCATION = "us-central1"

client = vertexai.Client(
    location=LOCATION,
)

## If you are using Agent Engine
See more documentation at [Agent Engine Overview](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/overview)

### Remote Testing

In [3]:
# Set to None to auto-detect from ./deployment_metadata.json, or specify manually
# "projects/PROJECT_ID/locations/us-central1/reasoningEngines/ENGINE_ID"
REASONING_ENGINE_ID = "projects/1045259343465/locations/us-central1/reasoningEngines/5491845076462075904"
#REASONING_ENGINE_ID = "projects/1045259343465/locations/asia-northeast1/reasoningEngines/6397134572261212160"

if REASONING_ENGINE_ID is None:
    try:
        with open("../deployment_metadata.json") as f:
            metadata = json.load(f)
            REASONING_ENGINE_ID = metadata.get("remote_agent_engine_id")
    except (FileNotFoundError, json.JSONDecodeError):
        pass

print(f"Using REASONING_ENGINE_ID: {REASONING_ENGINE_ID}")
# Get the existing agent engine
remote_agent_engine = client.agent_engines.get(name=REASONING_ENGINE_ID)

Using REASONING_ENGINE_ID: projects/1045259343465/locations/us-central1/reasoningEngines/5491845076462075904


In [4]:
session = await remote_agent_engine.async_create_session(user_id="test")
session

{'id': '490098786866036736',
 'appName': '5491845076462075904',
 'events': [],
 'lastUpdateTime': 1762309012.4585,
 'userId': 'test',
 'state': {}}

In [5]:
async for event in remote_agent_engine.async_stream_query(
    message="hi! what's the weather in San francisco?", user_id="test", session_id = session['id']
):
    print(event)

{'content': {'parts': [{'thought_signature': 'CugBAePx_164UrGPIpBZ5uW_xrffkMid8uZGx-MCMaEnXCLaLiCR-_MgIn0o9mJQPG45tFC1tD29XbV6Gj0SJrdS_sSC97vkiEh22BZ_PwWF86L7QVf_c5ysa7SeWEWeVtxSBjdjCPb_I2qC4ABnWyu3DakpU9J5CIn3gYSQi01F9O0MNzIokpduKYc0s5coZMMezZb_TGzANyLgQr1Tx4GZ3KL5g-ECkdHlvbkh9SuyFx88XZQUQqdVM8lVkasr08l8HWNzuOw_TohDexgcb1SINe0liAfv9nhPFN-A5Nb64uYHa3eVJXI0Bw==', 'function_call': {'id': 'adk-84218720-cecc-47e0-892b-3fda59d68358', 'args': {'query': 'San Francisco'}, 'name': 'get_weather'}}], 'role': 'model'}, 'finish_reason': 'STOP', 'usage_metadata': {'candidates_token_count': 6, 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 6}], 'prompt_token_count': 156, 'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 156}], 'thoughts_token_count': 47, 'total_token_count': 209, 'traffic_type': 'ON_DEMAND'}, 'avg_logprobs': -1.4913775126139324, 'invocation_id': 'e-f545b04a-b6b4-4dcb-87b0-568539a8db24', 'author': 'root_agent', 'actions': {'state_delta': {}, 'artifact_de

In [6]:
async for event in remote_agent_engine.async_stream_query(
    message="hi! I'd like to travel SFO", user_id="test", session_id = session['id']
):
    print(event)

{'content': {'parts': [{'thought_signature': 'CtoDAePx_16gndZIJRMeAUX26GhN8LitoFIYQT0U_JhK6nNVIL3pBYS5RE5GvxyjHzlYNffKmbqVrdqmkPnkskOBE2CMj1hhoxqQGb62PAIXiOBg4VkOshdFPZcI0k5NKTWCKFcM_A4AfTPhNrwpnCjVTHW4XttURJ7vnDk-hwq_PhMFOCRsdos3YdvAcmz9G_2RPEMm7McjYgEirTgqs0vJdir0zK9PQoM9rxUvUHvXLrmkkXKF2bcL-kbkSWq2Us2VmxAQr3IcWx674mKxmqT-Jfqwr3J8_CpJngHCEZQt21yg7BXyhSJGIp_hQXNhEb6DV7Is-dBuZAHGvlP4fIrWSqFe6Vdn9wMkots3_1BHcMHrsnXANqFmvTlyRaOKW8AYFj5Jy_R1-tBTo1Kt3QZkohlyRnp1-OVA79n8VGbYldlHT0WysscuJ6My0fQ_295fBD-iGAx_cq96wYg_DPOWYQd4fsxb7SRKPNsdnng-gL1v9ha8KW9d0TxYM53uoq9XuJ6nhlH0bR-bfYSevd0wrwavLai3CopxAuFhXsOkBriwRPkQb8odV96BeUCsA6iLC0DwbgCGvCg_bBCbNlqtpKg9M46_1BKnmGJcMqJyGA5Qk2SowErm44v1', 'text': "I can't help you with travel arrangements. I can tell you the weather or current time for a city."}], 'role': 'model'}, 'finish_reason': 'STOP', 'usage_metadata': {'candidates_token_count': 23, 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 23}], 'prompt_token_count': 199, 'prompt_token

In [7]:
session = await remote_agent_engine.async_get_session(user_id="test", session_id=session['id'])
#session

In [11]:
await remote_agent_engine.async_add_session_to_memory(session=session)

In [12]:
await remote_agent_engine.async_search_memory(user_id="test", query="I want to travel SFO")

{'memories': []}

In [ ]:
events =  [
  {
    "content": {
      "role": "user",
      "parts": [
        {"text": "I work with LLM agents!"}
      ]
    }
  }
]

remote_agent_engine.memories.generate(
    name=remote_agent_engine.api_resource.name,
    direct_contents_source={
      "events": events
    },
    # For example, `scope={"user_id": "123"}`.
    scope={"user_id": "test"},
    config={
        "wait_for_completion": True
    }
)

In [ ]:
remote_agent_engine.register_feedback(
    feedback={
        "score": 5,
        "text": "Great response!",
        "invocation_id": "test-invocation-123",
        "user_id": "test",
    }
)

### Local Testing

You can import directly the AgentEngineApp class within your environment. 
To run the agent locally, follow these steps:
1. Make sure all required packages are installed in your environment
2. The recommended approach is to use the same virtual environment created by the 'uv' tool
3. You can set up this environment by running 'make install' from your agent's root directory
4. Then select this kernel (.venv folder in your project) in your Jupyter notebook to ensure all dependencies are available

In [ ]:
# Uncomment the following lines if you're not using the virtual environment created by uv
# import sys
# sys.path.append("../")

In [ ]:
from app.agent import root_agent
from app.agent_engine_app import AgentEngineApp

agent_engine = AgentEngineApp(agent=root_agent)

In [ ]:
async for event in agent_engine.async_stream_query(message="hi!", user_id="test"):
    print(event)

## If you are using Cloud Run

#### Remote Testing

For more information about authenticating HTTPS requests to Cloud Run services, see:
[Cloud Run Authentication Documentation](https://cloud.google.com/run/docs/triggering/https-request)

Remote testing involves using a deployed service URL instead of localhost.

Authentication is handled using GCP identity tokens instead of local credentials.

In [ ]:
ID_TOKEN = get_ipython().getoutput("gcloud auth print-identity-token -q")[0]

In [ ]:
SERVICE_URL = "YOUR_SERVICE_URL_HERE"  # Replace with your Cloud Run service URL

You'll need to first create a Session

In [ ]:
user_id = "test_user_123"
session_data = {"state": {"preferred_language": "English", "visit_count": 1}}

session_url = f"{SERVICE_URL}/apps/app/users/{user_id}/sessions"
headers = {"Content-Type": "application/json", "Authorization": f"Bearer {ID_TOKEN}"}

session_response = requests.post(session_url, headers=headers, json=session_data)
print(f"Session creation status code: {session_response.status_code}")
print(f"Session creation response: {session_response.json()}")
session_id = session_response.json()["id"]

Then you will be able to send a message

In [ ]:
message_data = {
    "app_name": "app",
    "user_id": user_id,
    "session_id": session_id,
    "new_message": {"role": "user", "parts": [{"text": "Hello! Weather in New york?"}]},
    "streaming": True,
}

message_url = f"{SERVICE_URL}/run_sse"
message_response = requests.post(
    message_url, headers=headers, json=message_data, stream=True
)

print(f"Message send status code: {message_response.status_code}")

# Print streamed response
for line in message_response.iter_lines():
    if line:
        line_str = line.decode("utf-8")
        if line_str.startswith("data: "):
            event_json = line_str[6:]
            event = json.loads(event_json)
            print(f"Received event: {event}")

### Local Testing

> You can run the application locally via the `make local-backend` command.

#### Create a session
 Create a new session with user preferences and state information


In [ ]:
user_id = "test_user_123"
session_data = {"state": {"preferred_language": "English", "visit_count": 1}}

session_url = f"http://127.0.0.1:8000/apps/app/users/{user_id}/sessions"
headers = {"Content-Type": "application/json"}

session_response = requests.post(session_url, headers=headers, json=session_data)
print(f"Session creation status code: {session_response.status_code}")
print(f"Session creation response: {session_response.json()}")
session_id = session_response.json()["id"]

#### Send a message
Send a message to the backend service and receive a streaming response


In [ ]:
message_data = {
    "app_name": "app",
    "user_id": user_id,
    "session_id": session_id,
    "new_message": {"role": "user", "parts": [{"text": "Hello! Weather in New york?"}]},
    "streaming": True,
}

message_url = "http://127.0.0.1:8000/run_sse"
message_response = requests.post(
    message_url, headers=headers, json=message_data, stream=True
)

print(f"Message send status code: {message_response.status_code}")

# Print streamed response
for line in message_response.iter_lines():
    if line:
        line_str = line.decode("utf-8")
        if line_str.startswith("data: "):
            event_json = line_str[6:]
            event = json.loads(event_json)
            print(f"Received event: {event}")

Delete agent

In [ ]:
client = vertexai.Client(
    location="us-central1",
)
client.agent_engines.delete(
    name="projects/1045259343465/locations/us-central1/reasoningEngines/8107380928801669120",
    force=True, # Optional, if the agent has resources (e.g. sessions, memory)
)